In [1]:
import os
import pickle
import sys
import math
sys.path.append("../")
from od.env.config import SUMO_SIM_SECONDS
from od.misc.statistic import NetFlowType
from od.network.types import ResourceAllocatorType,BaseStationType
from od.misc.interest import InterestConfig
from od.env.config import SUMO_SIM_STEPS,ROOT_DIR
from od.env.station import BS_PRESET
from od.social.manager.types import DynamicSocialGroupBehaviour
from od.env.config import NET_TIMEOUT_SECONDS
import od.misc.statistic as oms

In [12]:
print(os.getcwd())
interest_config = InterestConfig(DynamicSocialGroupBehaviour.MAX_N_MEMBER,20, True,ResourceAllocatorType.NOMA_APR,True,1.3,7)
statistic = None
print(interest_config)
with open("../data/GS20/{}/object.pickle".format(interest_config.folder_legacy(1)), "rb") as file:
    statistic = pickle.load(file)
statistic.nft_traffic = {}
statistic.raw_sg_header = statistic.sg_header

/home/OD/repos/ProjectSumo/ipynb
MAX_N_MEMBER(20)-yQoS-yRSU-NOMA_APR-D1.3-S7


In [13]:

BS_CONFIG =  {
        name:bs_config 
        for name,bs_config in BS_PRESET.items()
        if (
            bs_config["type"] == BaseStationType.UMA  or 
            (
                interest_config.req_rsu and 
                bs_config["type"] == BaseStationType.UMI
            )
        )
}

class FakeBaseStation:
    def __init__(self,name,pos,bs_type,serial):
        self.name = name
        self.pos = pos
        self.type = bs_type
        self.serial = serial
    def __repr__(self) -> str:
        return self.name
    def __index__(self):
        return self.serial
        
class GV:
    NET_STATION_CONTROLLER = [ 
        FakeBaseStation(
            name,
            BS_CONFIG[name]["pos"],
            BS_CONFIG[name]["type"] ,
            i
        )   for i,name in enumerate(BS_CONFIG)
    ]
    class STATISTIC:
        def Log(data):
            pass
        def Doc(data):
            pass
    class RESULT:
        def Log(data):
            pass
        def Doc(data):
            print(data)
oms.GV = GV


In [14]:
GV.NET_STATION_CONTROLLER

[UMA-1, UMA-2, UMI-1, UMI-2, UMI-3, UMI-4, UMI-5]

In [15]:
TARGET_NFT = NetFlowType.C2G

In [16]:
app_stats = statistic.ExtractNetworkTraffic(TARGET_NFT,True)

In [17]:
statistic.BaseStationTypeThroughPutReport(app_stats, BaseStationType.UMA)

{'bits': 60787248, 'count': 10890}

In [18]:
for sg in statistic.social_group:
    print(statistic.AppdataTimeoutRatioReport(statistic.sg_header[sg]))

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.6455026455026455


In [19]:
# preprocess overtimes
for sg in statistic.social_group:
    for record in statistic.sg_header[sg].values():
        # ignore appdata that're flagged overtime.
        if(record.is_src_ot):
            continue
        else:
            # Check data source overtime condition.
            in_txq_time = list(filter(lambda x: len(x) > 0, record.time_bs_txq))
            min_enter_txq_time = min(
                list(map(
                    lambda x: x[0][0],
                    in_txq_time
                )) + [NET_TIMEOUT_SECONDS + record.at]
            )
            if(len(in_txq_time) == 0 or
                min_enter_txq_time - record.at >= NET_TIMEOUT_SECONDS):
                record.is_src_ot = True
            else:
                # Check base station overtime condition.
                for bs in GV.NET_STATION_CONTROLLER:
                    if(
                        len(record.time_bs_txq[bs]) > 0 and
                        record.time_bs_txq[bs][-1][1] - record.at >= NET_TIMEOUT_SECONDS
                    ):
                        record.is_bs_ot[bs] = True
statistic.social_group


[<CRASH(QoS:0, Gid:0)>,
 <RCWS(QoS:1, Gid:1)>,
 <DYN_SG_0(QoS:1, Gid:2)>,
 <DYN_SG_1(QoS:1, Gid:3)>,
 <DYN_SG_2(QoS:1, Gid:4)>,
 <DYN_SG_3(QoS:1, Gid:5)>,
 <DYN_SG_4(QoS:1, Gid:6)>,
 <DYN_SG_5(QoS:1, Gid:7)>,
 <DYN_SG_6(QoS:1, Gid:8)>,
 <DYN_SG_7(QoS:1, Gid:9)>,
 <DYN_SG_8(QoS:1, Gid:10)>]

In [23]:
print(type(statistic.sg_header[sg]))

for sg in statistic.social_group:
    print(statistic.AppdataTimeoutRatioReport(statistic.sg_header[sg]))

<class 'dict'>
0.0
0.0
0.0
0.0
0.0
0.7139256458727158
0.0
0.0
0.0
0.0
1.0
